In [6]:
# Data file at https://www.cse.ust.hk/msbd5003/data

lines = sc.textFile('../data/adj_noun_pairs.txt', 8)

In [7]:
lines.count()

3162692

In [8]:
lines.getNumPartitions()

8

In [9]:
lines.take(5)

['early radical',
 'french revolution',
 'pejorative way',
 'violent means',
 'positive label']

In [10]:
# Converting lines into word pairs. 
# Data is dirty: some lines have more than 2 words, so filter them out.
pairs = lines.map(lambda l: tuple(l.split())).filter(lambda p: len(p)==2)
pairs.cache()

PythonRDD[11] at RDD at PythonRDD.scala:53

In [11]:
pairs.take(5)

[('early', 'radical'),
 ('french', 'revolution'),
 ('pejorative', 'way'),
 ('violent', 'means'),
 ('positive', 'label')]

In [12]:
N = pairs.count()

In [13]:
N

3162674

In [14]:
# Compute the frequency of each pair.
# Ignore pairs that not frequent enough
pair_freqs = pairs.map(lambda p: (p,1)).reduceByKey(lambda f1, f2: f1 + f2) \
                  .filter(lambda pf: pf[1] >= 100)

In [15]:
pair_freqs.take(5)

[(('16th', 'century'), 950),
 (('civil', 'war'), 2236),
 (('social', 'class'), 155),
 (('sixteenth', 'century'), 137),
 (('late', '1970'), 444)]

In [16]:
# Computing the frequencies of the adjectives and the nouns
a_freqs = pairs.map(lambda p: (p[0],1)).reduceByKey(lambda x,y: x+y)
n_freqs = pairs.map(lambda p: (p[1],1)).reduceByKey(lambda x,y: x+y)

In [17]:
a_freqs.take(5)

[('social', 7413),
 ('free', 6635),
 ('stoic', 72),
 ('good', 7499),
 ('other', 75260)]

In [20]:
n_freqs.count()

106333

In [21]:
# Broadcasting the adjective and noun frequencies. 
#a_dict = a_freqs.collectAsMap()
#a_dict = sc.parallelize(a_dict).map(lambda x: x)
n_dict = sc.broadcast(n_freqs.collectAsMap())
a_dict = sc.broadcast(a_freqs.collectAsMap())
a_dict.value['violent']

1191

In [22]:
from math import *

# Computing the PMI for a pair.
def pmi_score(pair_freq):
    w1, w2 = pair_freq[0]
    f = pair_freq[1]
    pmi = log(float(f)*N/(a_dict.value[w1]*n_dict.value[w2]), 2)
    return pmi, (w1, w2)

In [23]:
# Computing the PMI for all pairs.
scored_pairs = pair_freqs.map(pmi_score)

In [24]:
# Printing the most strongly associated pairs. 
scored_pairs.top(10)

[(14.41018838546462, ('magna', 'carta')),
 (13.071365888694997, ('polish-lithuanian', 'Commonwealth')),
 (12.990597616733414, ('nitrous', 'oxide')),
 (12.64972604311254, ('latter-day', 'Saints')),
 (12.50658937509916, ('stainless', 'steel')),
 (12.482331020687814, ('pave', 'runway')),
 (12.19140721768055, ('corporal', 'punishment')),
 (12.183248694293388, ('capital', 'punishment')),
 (12.147015483562537, ('rush', 'yard')),
 (12.109945794428935, ('globular', 'cluster'))]